In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

mnist = np.load("dataset/mnist-784-euclidean.npy")
print(f'mnist data shape: {mnist.shape}')
print(f'mnist dtype: {mnist.dtype}')

# mnist = np.random.uniform(low=0, high=255, size=(100000, 2))
# mnist = mnist.astype(np.float32)
# print(f'mnist dtype: {mnist.dtype}')
# print(f'mnist data shape: {mnist.shape}')


mnist data shape: (60000, 784)
mnist dtype: float32


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[    0 32248  8728 18932 30483 24149 42338 52295 26251 50173 53634 24330
  54159 57528  1482 53428 18123 31379 52864 10536 29719 36087 30489 23947
  20034 52057 33825 21654 31008 55208 22477 44282 47968 54203 19825  1634
  27378 33909 15378 24708 34474 26413 16017 46824 46358  1516 34557 16832
  21629 29021 10740 24107  5688 52665  1864  5036 39031  1978 40546 22322
  52231 37284 24730  5970 21976 16945  9568 36697 25675 54189 11396 42555
  33445 52540 44263 18404 19186 24232 54184 25762 14736 33970  5210 59212
   8642 22569 15052  2933  6772 22963  6516   832 21244 21583 35838 59846
  21210 13502 52559 13862]]
CPU times: total: 78.1 ms
Wall time: 70.9 ms


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

idx = index.Indexing(
    gl_size=10000, 
    ll_size=3000,
    g_epoch_num=5,
    l_epoch_num=10,
    g_hidden_size=10,
    l_hidden_size=20,
    g_block_range=4,
    l_block_range=4,
    random_partitioning=False
)
idx.train(mnist_tensor)

print(f"local model len:{len(idx._l_model)}")

mnist tensor shape: torch.Size([60000, 784])
torch.Size([1995, 785])
torch.Size([8906, 785])
torch.Size([4610, 785])
torch.Size([3738, 785])
torch.Size([7329, 785])
torch.Size([4915, 785])
torch.Size([8447, 785])
torch.Size([4660, 785])
torch.Size([2740, 785])
torch.Size([2620, 785])
torch.Size([382, 785])
torch.Size([9658, 785])
first stage partitioning finish
partitioning blocks : 12
training local model
1, 10 loss: -0.0
2, 10 loss: -0.0
3, 10 loss: -0.0
4, 10 loss: -0.0
5, 10 loss: -0.0


d:\conda\envs\ml\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


6, 10 loss: -0.0
7, 10 loss: -0.0
8, 10 loss: -0.0
9, 10 loss: -0.0
10, 10 loss: -0.0
training local model
1, 10 loss: 0.008268425464630127
1, 20 loss: 0.007569401264190674
1, 30 loss: 0.006799876689910889
1, 40 loss: 0.00661792516708374
2, 10 loss: 0.0064057749509811406
2, 20 loss: 0.006753825545310974
2, 30 loss: 0.006496251821517945
2, 40 loss: 0.006218312382698059
3, 10 loss: 0.006364407539367675
3, 20 loss: 0.006230348348617553
3, 30 loss: 0.006246569156646728
3, 40 loss: 0.006174065470695496
4, 10 loss: 0.0061866134405136105
4, 20 loss: 0.0059846508502960204
4, 30 loss: 0.005865477919578552
4, 40 loss: 0.006033384203910828
5, 10 loss: 0.005967954993247986
5, 20 loss: 0.006024157404899597
5, 30 loss: 0.006035679578781128
5, 40 loss: 0.006016644239425659
6, 10 loss: 0.005849908590316772
6, 20 loss: 0.006007859110832214
6, 30 loss: 0.006205464005470276
6, 40 loss: 0.005943137407302857
7, 10 loss: 0.005731755495071411
7, 20 loss: 0.0060200560092926025
7, 30 loss: 0.005847260951995849

d:\study\bspli\./Bspli\utils\Partitioning.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size

qp = torch.from_numpy(mnist[0])
# print(qp)
pred = idx.query(qp, k=100)
print(f"recall: {recall(pred, FLAT_I)}")

pred = pred.to(torch.int)
print(f"pred: {pred}")

print(f"isin same block: {idx.isin(qp)}")

recall: 0.17
pred: tensor([ 8728, 52295, 23947, 21654, 44282, 54203, 24708, 46824, 24107,  5036,
        22322, 21976, 36697, 25675, 25762, 14736, 22569, 43997, 53812, 15975,
        24271, 15907, 15444,  2395, 49334, 29023, 26364, 36242, 25557,  8230,
        24017, 11221, 45432, 22542, 33607, 43067, 25623, 54283, 52310, 42971,
        24281,  2037, 25720,  6274, 38240, 21144, 47090, 15482, 26328, 16013,
        17608,  2038, 57765, 24119, 20439, 55151, 48965, 59352, 20018, 55325,
        23341, 24651, 11097, 17366, 43103, 54281, 29112,  6513, 25508, 26526,
        28418, 44987, 17544, 47414,  4969, 18366, 41765, 50853, 54849, 49376,
        42447,  8766, 54366,   130, 37886, 54704, 39325, 36609,  9800, 54914,
        33407, 31230,  9932, 27482,  1718, 17504, 24574,  7640, 34893,  9646],
       dtype=torch.int32)
isin same block: False
CPU times: total: 328 ms
Wall time: 63.9 ms


In [5]:
%%time
pred = idx.query_with_threshold(qp, k=100, threshold=0.02, g_block_range=10)
print(f"recall: {recall(pred, FLAT_I)}")
pred = pred.to(torch.int)
print(f"pred: {pred}")

recall: 0.95
pred: tensor([    0, 32248,  8728, 18932, 30483, 24149, 42338, 52295, 26251, 50173,
        53634, 24330, 54159, 57528,  1482, 53428, 18123, 31379, 52864, 10536,
        36087, 30489, 23947, 20034, 52057, 33825, 21654, 31008, 55208, 22477,
        44282, 47968, 54203, 19825,  1634, 27378, 33909, 15378, 24708, 34474,
        26413, 16017, 46824, 46358,  1516, 34557, 21629, 29021, 10740, 24107,
         5688, 52665,  1864,  5036, 39031,  1978, 40546, 22322, 52231, 37284,
         5970, 21976, 16945, 36697, 25675, 54189, 11396, 42555, 33445, 52540,
        44263, 18404, 24232, 54184, 25762, 14736, 33970,  5210, 59212,  8642,
        22569, 15052,  2933,  6772, 22963,  6516,   832, 21244, 21583, 35838,
        59846, 21210, 13502, 52559, 13862, 18172, 41980, 43997, 53812, 18162],
       dtype=torch.int32)
CPU times: total: 641 ms
Wall time: 109 ms


In [6]:
result = []
indices = np.random.choice(mnist.shape[0], size=1000, replace=False)

def benchmark_knn_query(data, indices, index, k=100):
    query_time = 0
    cur_recall = 0
    isin_count = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100, g_block_range=7, l_block_range=4)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
        # isin_count += 1 if index.isin(q) else 0
        
    result.append((query_time/1000, cur_recall/1000, isin_count/1000))

In [8]:
# %%script false --no-raise-error
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, indices, idx, k=100)
print(result)
print(idx.get_search_blocks_num())

[(0.0794475429058075, 0.9228800000000033, 0.0)]
41


In [9]:
result = []

def benchmark_knn_query_with_threshold(data, indices, index, k=100):
    query_time = 0
    cur_recall = 0
    isin_count = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query_with_threshold(q, k=100, threshold=0.05, g_block_range=12)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
        # isin_count += 1 if index.isin(q) else 0
        
    result.append((query_time/1000, cur_recall/1000, isin_count/1000))



In [11]:
# %%script false --no-raise-error
benchmark_knn_query_with_threshold(mnist, indices, idx, k=100)

print(result)

[(0.060861944675445555, 0.8917400000000021, 0.0)]
